In [ ]:
import streamlit as st
import mysql.connector

# Create database
def create_db():
    conn = mysql.connector.connect("users.db")
    c = conn.cursor()
    c.execute('''
        CREATE TABLE IF NOT EXISTS users (
            username TEXT PRIMARY KEY,
            password TEXT NOT NULL
        )
    ''')
    conn.commit()
    conn.close()

create_db()

# Function to register a new user
def register(username, password):
    conn = mysql.connector.connect("users.db")
    c = conn.cursor()
    try:
        c.execute("INSERT INTO users (username, password) VALUES (?, ?)", (username, password))
        conn.commit()
        st.success("User registered successfully!")
    except mysql.connector.IntegrityError:
        st.error("Username already exists.")
    finally:
        conn.close()

# Function to check login credentials
def check_login(username, password):
    conn = mysql.connector.connect("users.db")
    c = conn.cursor()
    c.execute("SELECT * FROM users WHERE username = ? AND password = ?", (username, password))
    result = c.fetchone()
    conn.close()
    return result is not None

# Initialize session state
if "logged_in" not in st.session_state:
    st.session_state.logged_in = False

# Login and logout functions
def login():
    username = st.text_input("Username")
    password = st.text_input("Password", type='password')
    
    if st.button("Log in"):
        if check_login(username, password):
            st.session_state.logged_in = True
            st.success("Logged in successfully!")
            st.session_state.username = username
        else:
            st.error("Invalid username or password.")
        st.experimental_rerun()

def logout():
    if st.button("Log out"):
        st.session_state.logged_in = False
        del st.session_state.username
        st.success("Logged out successfully!")
        st.experimental_rerun()

# UI logic
if st.session_state.logged_in:
    st.write(f"Welcome, {st.session_state.username}!")
    logout()
else:
    st.header("Please log in")
    login()

    st.subheader("Register a new account")
    reg_username = st.text_input("New Username", "")
    reg_password = st.text_input("New Password", type='password')
    if st.button("Register"):
        register(reg_username, reg_password)
